## Imports

In [1]:
%matplotlib inline

import os
import glob
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import pylab as plt

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import random
import time
import collections

from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.svm import LinearSVC

## Lesson functions

In the lesson_functions, contain all the functions defined during the lesson of the Udacity class.
  * get_hog_features()
  * bin_spatial()
  * color_hist()
  * extract_features()
  * slide_window()
  * draw_boxes()

# Debug Class

In [2]:
# Debug class - used to control debug outputs during execution in a convenient way 
class Debug():
    def __init__(self, 
                 my_debug_write = False, 
                 my_debug_plot = False,                  
                 my_image_output_folder='test_images_output/',
                 my_image_filename_post='',    
                ):
        
        self.debug_write = my_debug_write;  
        self.debug_plot = my_debug_plot; 
        self.image_output_folder = my_image_output_folder;
        self.image_filename_post = my_image_filename_post; # post_filename for image_output
        
        if not os.path.exists(self.image_output_folder):
            os.makedirs(self.image_output_folder)
        
    def my_debug_out(self, img, shortname, is_gray=False, colorspace='RGB'):
        if (self.debug_plot): # Plot Image on screen
            plt.figure()
            if(is_gray == True):
                plt.imshow(img, cmap='gray')
            else:
                plt.imshow(img)
            plt.title(shortname, fontsize=25)
            
        # Write image to output_folder
        if (self.debug_write):
            if(is_gray == False):
                cv2.imwrite(self.image_output_folder  + shortname + self.image_filename_post + '.jpg', cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
            else:
                cv2.imwrite(self.image_output_folder  + shortname + self.image_filename_post + '.jpg', img)
                
    def my_histogram_out(self, hist, bin_centers, shortname):
        if (self.debug_plot == True or self.debug_write == True):
            fig = plt.figure()
            plt.bar(bin_centers, hist[0], width=2)
            plt.xlim(0, 256)
            plt.title(shortname)
            
            # Write image to output_folder
            if (self.debug_write):
                plt.savefig(self.image_output_folder  + shortname + self.image_filename_post + '.jpg')

            if (self.debug_plot == False): # Plot Image on screen
                plt.close(fig)

## Feature Extractor Class

In [3]:
# Class used to extract features from the training images and providing methods 

class FeatureExtractor():
    def __init__(self):
        
        self.my_debug = Debug(my_debug_write = False,
                              my_debug_plot = False,
                              my_image_output_folder='test_images_output/')
        self.color_space = 'RGB'
        self.spatial_size= (32, 32)
        self.hist_bins=32
        self.orient=9
        self.pix_per_cell=8
        self.cell_per_block=2
        self.hog_channel=0
        self.transform_sqrt=False
        self.spatial_feat=True
        self.hist_feat=True
        self.hog_feat=True
        
    # Based on code from Udacity Online Class
    # Define a function to return HOG features and visualization
    def get_hog_features(self, img, visualize = False, feature_vector=True):
        # Call with two outputs if vis==True
       
        if (visualize):
            features, hog_image = hog(img, orientations=self.orient, 
                                      pixels_per_cell=(self.pix_per_cell, self.pix_per_cell),
                                      cells_per_block=(self.cell_per_block, self.cell_per_block), 
                                      transform_sqrt=self.transform_sqrt, 
                                      visualise=visualize, feature_vector=feature_vector)
            return features, hog_image
        else:
            features = hog(img, orientations=self.orient, 
                            pixels_per_cell=(self.pix_per_cell, self.pix_per_cell),
                            cells_per_block=(self.cell_per_block, self.cell_per_block), 
                            transform_sqrt=self.transform_sqrt, 
                            visualise=visualize, feature_vector=feature_vector)
            return features
        

    # Based on code from Udacity Online Class
    def bin_spatial(self, img):
        # Use cv2.resize().ravel() to create the feature vector
        bin_spatial_img = cv2.resize(img, self.spatial_size)
        features = bin_spatial_img.ravel()
        # Return the feature vector
        return features, bin_spatial_img

    # Based on code from Udacity Online Class
    def color_hist(self, img):
        # Compute the histogram of the color channels separately
        channel1_hist = np.histogram(img[:,:,0], bins=self.hist_bins, range=(0, 256))
        channel2_hist = np.histogram(img[:,:,1], bins=self.hist_bins, range=(0, 256))
        channel3_hist = np.histogram(img[:,:,2], bins=self.hist_bins, range=(0, 256))
        # Concatenate the histograms into a single feature vector
        hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))

        # Generating bin centers
        bin_edges = channel1_hist[1]
        bin_centers = (bin_edges[1:]  + bin_edges[0:len(bin_edges)-1])/2

        return hist_features, channel1_hist, channel2_hist, channel3_hist, bin_centers
    
    # Based on code from Udacity Online Class
    def convert_color(self, img):
        if self.color_space == 'RGB':
            feature_image = img.copy()
        elif self.color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif self.color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif self.color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif self.color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif self.color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
        else:
            assert(False, 'Problem')
        return feature_image
    
    
    # Based on code from Udacity Online Class
    # This function is very similar to extract_features()
    # just for a single image rather than list of images
    def single_img_features(self, img):    
        
        self.my_debug.my_debug_out(img, shortname='01_input_img', is_gray=False)
        
        #1) Define an empty list to receive features
        img_features = []
        #2) Apply color conversion 
        feature_image = self.convert_color(img)
        
        self.my_debug.my_debug_out(feature_image[:,:,0], shortname='01_input_ch0', is_gray=True)
        self.my_debug.my_debug_out(feature_image[:,:,1], shortname='01_input_ch1', is_gray=True)
        self.my_debug.my_debug_out(feature_image[:,:,2], shortname='01_input_ch2', is_gray=True)    
            
        #3) Compute spatial features if flag is set
        if self.spatial_feat == True:
            spatial_features, bin_spatial_img = self.bin_spatial(feature_image)
            self.my_debug.my_debug_out(bin_spatial_img[:,:,0], shortname='02_bin_spatial_ch0', is_gray=True)
            self.my_debug.my_debug_out(bin_spatial_img[:,:,1], shortname='02_bin_spatial_ch1', is_gray=True)
            self.my_debug.my_debug_out(bin_spatial_img[:,:,2], shortname='02_bin_spatial_ch2', is_gray=True)            
            #4) Append features to list
            img_features.append(spatial_features)
            
        #5) Compute histogram features if flag is set
        if self.hist_feat == True:
            hist_features, ch1_hist, ch2_hist, ch3_hist, bin_centers = self.color_hist(feature_image)
            
            self.my_debug.my_histogram_out(ch1_hist, bin_centers, "03_hist_ch0")
            self.my_debug.my_histogram_out(ch2_hist, bin_centers, "03_hist_ch1")
            self.my_debug.my_histogram_out(ch3_hist, bin_centers, "03_hist_ch2")
            #6) Append features to list
            img_features.append(hist_features)
            
        #7) Compute HOG features if flag is set
        if self.hog_feat == True:
            if self.hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    ch_features, hog_image_float  = self.get_hog_features(feature_image[:,:,channel], visualize=True, 
                                                                          feature_vector=True)
                    self.my_debug.my_debug_out(hog_image_float*255, shortname='04_hog_ch'+ str(channel), is_gray=True)   
                    hog_features.extend(ch_features)      
            else:
                hog_features, hog_image_float = self.get_hog_features(feature_image[:,:,self.hog_channel], visualize=True, 
                                                                      feature_vector=True)
                self.my_debug.my_debug_out(hog_image_float*255, shortname='04_hog_ch'+ str(self.hog_channel), is_gray=True)   
                
            #8) Append features to list
            img_features.append(hog_features)


        #9) Return concatenated array of features
        return np.concatenate(img_features)
    
    # Based on code from Udacity Online Class
    def extract_features(self, imgs):
        # Create a list to append feature vectors to
        features = []
        # Iterate through the list of images
        for file in imgs:
            file_features = []
            
            # Save filename in order to save files
            self.my_debug.image_filename_post = '_' + os.path.splitext(os.path.basename(file))[0]
            
            # Read in each one by one
            image_in = cv2.imread(file) # BGR
            image_in = cv2.cvtColor(image_in, cv2.COLOR_BGR2RGB)
            
            file_features = self.single_img_features(image_in)   
            
            features.append(file_features)
        # Return list of feature vectors
        return features

    # From Udacity Online Class
    def normalize_features(self, img_features):
        X = np.vstack(img_features).astype(np.float64)                        
        # Fit a per-column scaler
        X_scaler = StandardScaler().fit(X)
        # Apply the scaler to X
        scaled_X = X_scaler.transform(X)
        
        return scaled_X, X_scaler
    
    # Based on provided function from Udacity Online Class
    # Define a single function that can extract features using hog sub-sampling and make predictions
    def find_cars(self, img, ystart, ystop, scale, svc, X_scaler):
        #time_start=time.time()
        #print('Hello World')
        
        draw_img = np.copy(img)

        img = img[ystart:ystop,:,:]

        ctrans_tosearch = self.convert_color(img)

        if scale != 1:
            imshape = ctrans_tosearch.shape
            ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))

        # Define blocks and steps as above
        nxblocks = (ctrans_tosearch[:,:,0].shape[1] // self.pix_per_cell) - self.cell_per_block + 1
        nyblocks = (ctrans_tosearch[:,:,0].shape[0] // self.pix_per_cell) - self.cell_per_block + 1 
        nfeat_per_block = self.orient*self.cell_per_block**2

        #print('nxblocks: ' + str(nxblocks))
        #print('nyblocks: ' + str(nyblocks))

        # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
        window = 64
        nblocks_per_window = (window // self.pix_per_cell) - self.cell_per_block + 1
        cells_per_step = 2  # Instead of overlap, define how many cells to step
        nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
        nysteps = (nyblocks - nblocks_per_window) // cells_per_step

        # Compute individual channel HOG features for the entire image
        
        
        if (self.hog_channel == 'ALL'):
            hog1 = self.get_hog_features(ctrans_tosearch[:,:,0], visualize=False, feature_vector=False)
            hog2 = self.get_hog_features(ctrans_tosearch[:,:,1], visualize=False, feature_vector=False)
            hog3 = self.get_hog_features(ctrans_tosearch[:,:,2], visualize=False, feature_vector=False)
        else:  # eiter 0,1,2
            hog_channel = self.get_hog_features(ctrans_tosearch[:,:,self.hog_channel], visualize=False, feature_vector=False)
        
        box_list = []

        #print(round(time.time()-time_start, 2), 's - before loop')
        
        for xb in range(nxsteps):
            for yb in range(nysteps):

                #print('  ',round(time.time()-time_start, 4), 's - loop start')
                ypos = yb*cells_per_step
                xpos = xb*cells_per_step
                
                
                # Extract HOG for this patch
                if (self.hog_channel == 'ALL'):
                    hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
                    hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
                    hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
                    hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))
                else:
                    hog_features = hog_channel[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
                    
                #print('  ',round(time.time()-time_start, 4), 's - after hog')
                
                xleft = xpos*self.pix_per_cell
                ytop = ypos*self.pix_per_cell

                # Extract the image patch
                subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))

                # Get color features
                #print('  ',round(time.time()-time_start, 4), 's - before features')
                spatial_features, _ = self.bin_spatial(subimg)
                hist_features, _, _, _, _   = self.color_hist(subimg)
                #print('  ',round(time.time()-time_start, 4), 's - after features')
                # Scale features and make a prediction            
                
                np.hstack((spatial_features, hist_features, hog_features))
                
                test_features = X_scaler.transform(np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1))    
                test_prediction = svc.predict(test_features)
                #print(test_prediction[0])
                #print('  ',round(time.time()-time_start, 4), 's - after prediction')

                if test_prediction == 1:
                    xbox_left = np.int(xleft*scale)
                    ytop_draw = np.int(ytop*scale)
                    win_draw = np.int(window*scale)
                    #cv2.rectangle(draw_img,(xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart),(0,0,255),6) 

                    box_list.append(((xbox_left, ytop_draw+ystart), (xbox_left+win_draw,ytop_draw+win_draw+ystart)))
                    
                    
        #print(round(time.time()-time_start, 2), 's - after loop')
        return box_list


# Vehicle Detector

In [23]:
class VehicleDetector():
    def __init__(self, svc, X_scaler, feature_extractor):
        self.svc = svc
        self.X_scaler = X_scaler
        self.feature_extractor = feature_extractor
        self.recent_heats = collections.deque(maxlen=10) 
        self.my_debug = Debug(my_debug_write = False,
                              my_debug_plot = False,
                              my_image_output_folder='test_images_output/')
    
    
    def process_image(self, image_in):
        
        self.my_debug.my_debug_out(img=image_in, shortname='01_image_in')
        
        
        ystart = 400
        ystop = 656
        
        scale = 1
        out_boxes = self.feature_extractor.find_cars(image_in, ystart, ystop, scale, self.svc, self.X_scaler)
        
        scale = 1.33
        out_boxes1 = self.feature_extractor.find_cars(image_in, ystart, ystop, scale, self.svc, self.X_scaler)
        out_boxes.extend(out_boxes1)
        
        scale = 1.66
        out_boxes2 = self.feature_extractor.find_cars(image_in, ystart, ystop, scale, self.svc, self.X_scaler)
        out_boxes.extend(out_boxes2)
        
        scale = 2
        out_boxes3 = self.feature_extractor.find_cars(image_in, ystart, ystop, scale, self.svc, self.X_scaler)
        out_boxes.extend(out_boxes3)
        
        image_boxes = self.draw_boxes(image_in, out_boxes)
        self.my_debug.my_debug_out(img=image_boxes, shortname='02_image_boxes', is_gray=False)

        heat = np.zeros_like(image_in[:,:,0]).astype(np.float)
        heat =  self.add_heat(heat,out_boxes)
        self.my_debug.my_debug_out(img=heat, shortname='03_heatmap', is_gray=True)
        heat = self.apply_threshold(heat,3)
        
        self.recent_heats.append(heat)
        
        heat = np.mean(my_vehicle_detector.recent_heats, axis=0)
        heat = self.apply_threshold(heat, 1)

        from scipy.ndimage.measurements import label
        heatmap = np.clip(heat, 0, 255)
        labels = label(heatmap)
        draw_img = self.draw_labeled_bboxes(np.copy(image_in), labels)
        
        self.my_debug.my_debug_out(img=draw_img, shortname='04_out_img', is_gray=False)
        
        return draw_img
    
    # From Udacity Online Class
    # Define a function to draw bounding boxes
    def draw_boxes(self, img, bboxes, color=(0, 0, 255), thick=6):
        # Make a copy of the image
        imcopy = np.copy(img)
        # Iterate through the bounding boxes
        for bbox in bboxes:
            # Draw a rectangle given bbox coordinates
            cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
        # Return the image copy with boxes drawn
        return imcopy
    
    # From Udacity Online Class
    def add_heat(self, heatmap, bbox_list):
        # Iterate through list of bboxes
        for box in bbox_list:
            # Add += 1 for all pixels inside each bbox
            # Assuming each "box" takes the form ((x1, y1), (x2, y2))
            heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

        # Return updated heatmap
        return heatmap
    
    # From Udacity Online Class
    def apply_threshold(self, heatmap, threshold):
        # Zero out pixels below the threshold
        heatmap[heatmap <= threshold] = 0
        heatmap[heatmap > 6] = 6
        # Return thresholded map
        return heatmap
    
    
    # From Udacity Online Class
    def draw_labeled_bboxes(self, img, labels):
        # Iterate through all detected cars
        for car_number in range(1, labels[1]+1):
            # Find pixels with each car_number label value
            nonzero = (labels[0] == car_number).nonzero()
            # Identify x and y values of those pixels
            nonzeroy = np.array(nonzero[0])
            nonzerox = np.array(nonzero[1])
            # Define a bounding box based on min/max x and y
            bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
            # Draw the box on the image
            cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
        # Return the image
        return img
    

# Prepare

In [5]:
# create Feature Extractor Object and define parameter

my_feature_extractor = FeatureExtractor() 

my_feature_extractor.my_debug.my_image_output_folder='test_images_output/'
my_feature_extractor.color_space = 'YCrCb'
my_feature_extractor.spatial_size= (16, 16)
my_feature_extractor.hist_bins=32
my_feature_extractor.orient=9
my_feature_extractor.pix_per_cell=8
my_feature_extractor.cell_per_block=2
my_feature_extractor.hog_channel='ALL'
my_feature_extractor.transform_sqrt=False
my_feature_extractor.spatial_feat=True
my_feature_extractor.hist_feat=True
my_feature_extractor.hog_feat=True

# Load Data

In [6]:
##### Load Training Data #### 
car_dirs = ["data/vehicles/KITTI_extracted/", "data/vehicles/GTI_Far/", "data/vehicles/GTI_Left/", 
            "data/vehicles/GTI_MiddleClose/", "data/vehicles/GTI_Right/"]
notcar_dirs = ["data/non-vehicles/GTI/", "data/non-vehicles/Extras/"]


car_images = []
for my_dir in car_dirs:
    car_images.extend(glob.glob(my_dir+'*.png'))

notcar_images = []
for my_dir in notcar_dirs:
    notcar_images.extend(glob.glob(my_dir+'*.png') ) 

# Shuffle the data within the image arrays
random.shuffle(car_images)
random.shuffle(notcar_images)

print('car_images length:', len(car_images))
print('notcar_images length:', len(notcar_images))

car_images length: 8792
notcar_images length: 8968


# Train Classifier

In [7]:
my_feature_extractor.my_debug.debug_plot = False;
my_feature_extractor.my_debug.debug_write = False;

# Reduce the sample size because
# The quiz evaluator times out after 13s of CPU time
#sample_size = 1000
#car_images = car_images[0:sample_size]
#notcar_images = notcar_images[0:sample_size]
    
    
###### FEATURE EXTRACTION #######
car_features = my_feature_extractor.extract_features(car_images)
notcar_features = my_feature_extractor.extract_features(notcar_images)

feature_list = [car_features, notcar_features] 

scaled_X, X_scaler = my_feature_extractor.normalize_features(feature_list)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=rand_state)

print('Feature vector length:', len(X_train[0]))

###### TRAIN CLASSIFIER #######
# Use a linear SVC 
svc = LinearSVC(verbose=True)
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))

Feature vector length: 6156
[LibLinear]41.25 Seconds to train SVC...
Test Accuracy of SVC =  0.9901


# Test Images

In [27]:
my_feature_extractor.my_debug.debug_plot = False;
my_feature_extractor.my_debug.debug_write = False;

test_images_dir = "test_images/"
test_images = glob.glob(test_images_dir+'*.jpg')  

#### Uncomment if only testing one image
#test_images = ['test_images/test6.jpg']  
    
for image_to_read in test_images:
    
    my_vehicle_detector = VehicleDetector(svc=svc, X_scaler=X_scaler, feature_extractor=my_feature_extractor)
    my_vehicle_detector.my_debug.debug_plot = False
    my_vehicle_detector.my_debug.debug_write = True

    filename_txt = os.path.splitext(os.path.basename(image_to_read))[0]
    my_vehicle_detector.my_debug.image_filename_post = '_'  + filename_txt
    
    image_in = cv2.imread(image_to_read)
    image_in = cv2.cvtColor(image_in, cv2.COLOR_BGR2RGB)
    im_out = my_vehicle_detector.process_image(image_in)
    

# Test Videos

In [28]:
my_vehicle_detector = VehicleDetector(svc=svc, X_scaler=X_scaler, feature_extractor=my_feature_extractor)
my_feature_extractor.my_debug.debug_plot = False;
my_feature_extractor.my_debug.debug_write = False;


if not os.path.exists('test_videos_output'):
    os.makedirs('test_videos_output')
white_output = 'test_videos_output/project_video.mp4'
    
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/project_video.mp4").subclip(18,22)
white_clip = clip1.fl_image(my_vehicle_detector.process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/project_video.mp4
[MoviePy] Writing video test_videos_output/project_video.mp4


 99%|█████████▉| 100/101 [03:49<00:02,  2.27s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/project_video.mp4 

CPU times: user 3min 49s, sys: 4.34 s, total: 3min 53s
Wall time: 3min 51s


In [ ]:
heat = my_vehicle_detector.recent_heats[-7]

plt.figure()
plt.imshow(heat)